In [1]:
import numpy as np
import sympy as sp
from scipy.linalg import eig

In [83]:
#Properties (N and mm)
E1 = 60800
E2 = 58250
v12 = 0.07
G12 = 4550
t = 0.21
v21 = v12 * E2 / E1

# Plate Parameters
a = 360
b = 360
m = 4
n = 4

# Loads
Nxx = 0
Nyy = 0
Nxy = -1

In [84]:
#Boundary conditions
u1tx = 0 ; u1rx = 1 ; u2tx = 0 ; u2rx = 1
v1tx = 0 ; v1rx = 1 ; v2tx = 0 ; v2rx = 1
w1tx = 0 ; w1rx = 1 ; w2tx = 0 ; w2rx = 1
u1ty = 0 ; u1ry = 1 ; u2ty = 0 ; u2ry = 1
v1ty = 0 ; v1ry = 1 ; v2ty = 0 ; v2ry = 1
w1ty = 0 ; w1ry = 1 ; w2ty = 0 ; w2ry = 1


**[Q] in the principal direction**  pg. 52 chap 2 of the three regions

In [85]:
Q_0 = np.array([[E1/(1-v12*v21), v12*E2/(1-v12*v21), 0],
                 [v12*E2/(1-v12*v21), E2/(1-v12*v21),0],
                 [0, 0, G12]])

**[Q] rotated in theta direction** pg. 53 and 54 of chap 2

In [86]:
layup = [(45,'f'),(0,'f'),(0,'f'),(45,'f'),(0,'f'),(0,'f'),(45,'f')]
Q_layup = []

#Layup assembly
for theta in layup:
    c = np.cos(theta[0]*np.pi/180)
    s = np.sin(theta[0]*np.pi/180)

    T_real = np.array([[c**2, s**2, 2*c*s],
                       [s**2, c**2, -2*c*s],
                       [-c*s, c*s, c**2-s**2]])

    T_engineering =  np.array([[c**2, s**2, c*s],
                               [s**2, c**2, -c*s],
                               [-2*c*s, 2*c*s, c**2-s**2]])


    if theta[1] == 'f':
        Q_layup.append((np.linalg.inv(T_real))@Q_0@T_engineering)


**[A], [B], [D]** pg. 24 fo chap 4 of the three regions

In [87]:
t_position = [-3.5*t, -2.5*t, -1.5*t, -0.5*t, 0.5*t, 1.5*t, 2.5*t, 3.5*t]

A = np.zeros(9).reshape(3,3)
B = np.zeros(9).reshape(3,3)
D = np.zeros(9).reshape(3,3)

for i in enumerate(Q_layup):
    zk1 = t_position[i[0]+1]
    zk0 = t_position[i[0]]
    
    A += (zk1 - zk0)*i[1]
    B += (1/2)*(zk1**2 - zk0**2)*i[1]
    D += (1/3)*(zk1**3 - zk0**3)*i[1]


F = np.vstack([
    np.hstack([A, B]),
    np.hstack([B, D])
])

F = sp.Matrix(F)

### Rayleigh-Ritz formulation

In [88]:
Ny = sp.symbols('Ny')
xi, eta  = sp.symbols(['xi', 'eta'])

In [89]:
xi**4/8 - xi**2/4 + 0.125

xi**4/8 - xi**2/4 + 0.125

In [90]:
# Bardell terms
# Su
uf_xi = {0: u1tx*(0.25*xi**3 - 0.75*xi + 0.5),
        1: u1rx*(0.125*xi**3 - 0.125*xi**2 - 0.125*xi + 0.125),
        2: u2tx*(-0.25*xi**3 + 0.75*xi + 0.5),
        3: u2rx*(-1/8 - 1/8*xi + 1/8*xi**2 + 1/8*xi**3),
        4: xi**4/8 - xi**2/4 + 0.125,
        5: xi**5/8 - xi**3/4 + xi/8,
        6: 7*xi**6/48 - 5*xi**4/16 + 3*xi**2/16 - 0.0208333333333333,
        7: 3*xi**7/16 - 7*xi**5/16 + 5*xi**3/16 - xi/16,
        8: 33*xi**8/128 - 21*xi**6/32 + 35*xi**4/64 - 5*xi**2/32 + 0.0078125,
        9: 143*xi**9/384 - 33*xi**7/32 + 63*xi**5/64 - 35*xi**3/96 + 5*xi/128}

uf_eta = {0: u1ty*(0.25*eta**3 - 0.75*eta + 0.5),
        1: u1ry*(0.125*eta**3 - 0.125*eta**2 - 0.125*eta + 0.125),
        2: u2ty*(-0.25*eta**3 + 0.75*eta + 0.5),
        3: u2ry*(-1/8 - 1/8*xi + 1/8*xi**2 + 1/8*xi**3),
        4: eta**4/8 - eta**2/4 + 0.125,
        5: eta**5/8 - eta**3/4 + eta/8,
        6: 7*eta**6/48 - 5*eta**4/16 + 3*eta**2/16 - 0.0208333333333333,
        7: 3*eta**7/16 - 7*eta**5/16 + 5*eta**3/16 - eta/16,
        8: 33*eta**8/128 - 21*eta**6/32 + 35*eta**4/64 - 5*eta**2/32 + 0.0078125,
        9: 143*eta**9/384 - 33*eta**7/32 + 63*eta**5/64 - 35*eta**3/96 + 5*eta/128}

# Sv
vf_xi = {0: v1tx*(0.25*xi**3 - 0.75*xi + 0.5),
        1: v1rx*(0.125*xi**3 - 0.125*xi**2 - 0.125*xi + 0.125),
        2: v2tx*(-0.25*xi**3 + 0.75*xi + 0.5),
        3: v2rx*(-1/8 - 1/8*xi + 1/8*xi**2 + 1/8*xi**3),
        4: xi**4/8 - xi**2/4 + 0.125,
        5: xi**5/8 - xi**3/4 + xi/8,
        6: 7*xi**6/48 - 5*xi**4/16 + 3*xi**2/16 - 0.0208333333333333,
        7: 3*xi**7/16 - 7*xi**5/16 + 5*xi**3/16 - xi/16,
        8: 33*xi**8/128 - 21*xi**6/32 + 35*xi**4/64 - 5*xi**2/32 + 0.0078125,
        9: 143*xi**9/384 - 33*xi**7/32 + 63*xi**5/64 - 35*xi**3/96 + 5*xi/128}

vf_eta = {0: v1ty*(0.25*eta**3 - 0.75*eta + 0.5),
        1: v1ry*(0.125*eta**3 - 0.125*eta**2 - 0.125*eta + 0.125),
        2: v2ty*(-0.25*eta**3 + 0.75*eta + 0.5),
        3: v2ry*(-1/8 - 1/8*xi + 1/8*xi**2 + 1/8*xi**3),
        4: eta**4/8 - eta**2/4 + 0.125,
        5: eta**5/8 - eta**3/4 + eta/8,
        6: 7*eta**6/48 - 5*eta**4/16 + 3*eta**2/16 - 0.0208333333333333,
        7: 3*eta**7/16 - 7*eta**5/16 + 5*eta**3/16 - eta/16,
        8: 33*eta**8/128 - 21*eta**6/32 + 35*eta**4/64 - 5*eta**2/32 + 0.0078125,
        9: 143*eta**9/384 - 33*eta**7/32 + 63*eta**5/64 - 35*eta**3/96 + 5*eta/128}

#Sw
wf_xi = {0: w1tx*(0.25*xi**3 - 0.75*xi + 0.5),
        1: w1rx*(0.125*xi**3 - 0.125*xi**2 - 0.125*xi + 0.125),
        2: w2tx*(-0.25*xi**3 + 0.75*xi + 0.5),
        3: w2rx*(-1/8 - 1/8*xi + 1/8*xi**2 + 1/8*xi**3),
        4: xi**4/8 - xi**2/4 + 0.125,
        5: xi**5/8 - xi**3/4 + xi/8,
        6: 7*xi**6/48 - 5*xi**4/16 + 3*xi**2/16 - 0.0208333333333333,
        7: 3*xi**7/16 - 7*xi**5/16 + 5*xi**3/16 - xi/16,
        8: 33*xi**8/128 - 21*xi**6/32 + 35*xi**4/64 - 5*xi**2/32 + 0.0078125,
        9: 143*xi**9/384 - 33*xi**7/32 + 63*xi**5/64 - 35*xi**3/96 + 5*xi/128}

wf_eta = {0: w1ty*(0.25*eta**3 - 0.75*eta + 0.5),
        1: w1ry*(0.125*eta**3 - 0.125*eta**2 - 0.125*eta + 0.125),
        2: w2ty*(-0.25*eta**3 + 0.75*eta + 0.5),
        3: w2ry*(-1/8 - 1/8*xi + 1/8*xi**2 + 1/8*xi**3),
        4: eta**4/8 - eta**2/4 + 0.125,
        5: eta**5/8 - eta**3/4 + eta/8,
        6: 7*eta**6/48 - 5*eta**4/16 + 3*eta**2/16 - 0.0208333333333333,
        7: 3*eta**7/16 - 7*eta**5/16 + 5*eta**3/16 - eta/16,
        8: 33*eta**8/128 - 21*eta**6/32 + 35*eta**4/64 - 5*eta**2/32 + 0.0078125,
        9: 143*eta**9/384 - 33*eta**7/32 + 63*eta**5/64 - 35*eta**3/96 + 5*eta/128}

In [91]:
Su, Sv, Sw = [[]], [[]], [[]]

for i in range(m):
    for j in range(n):
        Su[0].append(uf_xi[i]*uf_eta[j])
        Sv[0].append(vf_xi[i]*vf_eta[j])
        Sw[0].append(wf_xi[i]*wf_eta[j])
Su, Sv, Sw = sp.Matrix(Su), sp.Matrix(Sv), sp.Matrix(Sw)


B0_11 = np.array((2/a) * sp.diff(Su, xi))
B0_22 = np.array((2/b) * sp.diff(Sv, eta))
B0_31 = np.array((2/b)*sp.diff(Su, eta))
B0_32 = np.array((2/a)*sp.diff(Sv, xi))
B0_43 = np.array(-(4/a**2) * sp.diff(Sw, xi, xi))
B0_53 = np.array(-(4/b**2) * sp.diff(Sw, eta, eta))
B0_63 = np.array(-2*(4/(a*b)) * sp.diff(Sw, xi, eta))
Z = np.zeros(m*n).reshape(1, m*n)

B0_kappa = sp.Matrix([
    np.hstack([*B0_11, *Z, *Z]),
    np.hstack([*Z, *B0_22, *Z]),
    np.hstack([*B0_31, *B0_32, *Z]),
    np.hstack([*Z, *Z, *B0_43]),
    np.hstack([*Z, *Z, *B0_53]),
    np.hstack([*Z, *Z, *B0_63])
])


F = sp.Matrix(F)


G = sp.Matrix([
    np.hstack([*Z, *Z, *sp.diff(Sw, xi)*(2/a)]),
    np.hstack([*Z, *Z, *sp.diff(Sw, eta)*(2/b)])
 ])


     

G = sp.Matrix(G)

N = sp.Matrix([[Nxx, Nxy],
                [Nxy, Nyy]])

In [92]:
G.shape

(2, 48)

In [93]:
KG = sp.integrate(
     sp.integrate(
        G.T * N * G, (xi, -1, 1)), (eta, -1, 1)
) * (a*b/4)

KG = np.array(KG, dtype=np.longdouble)

In [94]:
print(KG.shape)
print(KG.min())
print(KG.max())

(48, 48)
-9.219424235271025e-19
9.219424235271025e-19


In [28]:
K = sp.integrate(
    sp.integrate(
        B0_kappa.T * F * B0_kappa, (xi, -1, 1)), (eta, -1, 1)
) * (a*b/4)

K = np.array(K, dtype=np.longdouble)

In [30]:
print(K.shape)
print(K.min())
print(K.max())

(48, 48)
-103.20648898856882
214.4550420541692


In [31]:
A = K
B = KG

eig_values, eig_vectors = eig(A, B)

In [32]:
eig_values.min()

(nan+0j)

In [33]:
eig_values

array([nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j,
       nan+0.j, nan+0.j, nan+0.j, nan+0.j, inf+0.j, inf+0.j, nan+0.j,
       nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j,
       nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j,
       nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j,
       nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j,
       nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j, nan+0.j])

In [36]:
eigvals, eigvecs = lb(A, B, silent=True)
eigvals

d:\programs\Anaconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:936: LinAlgWarning: Diagonal number 1 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
d:\programs\Anaconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1593: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "


TypeError: Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.

## Check with compmech

In [37]:
from compmech.panel.panel import Panel
from compmech.analysis import lb, static

In [76]:
# skin panels
laminaprop = (E1, E2, v12, G12, G12, G12)
stack = [45, 0, 0, 45, 0, 0, 45,]
p1 = Panel(group='skin', Nxy=-1, x0=0, y0=0., a=a, b=b,m=4, n=4, plyt=t, stack=stack, laminaprop=laminaprop)

In [77]:
# boundary conditions CCCC
p1.u1tx = 0 ; p1.u1rx = 1 ; p1.u2tx = 0 ; p1.u2rx = 1
p1.v1tx = 0 ; p1.v1rx = 1 ; p1.v2tx = 0 ; p1.v2rx = 1
p1.w1tx = 0 ; p1.w1rx = 1 ; p1.w2tx = 0 ; p1.w2rx = 1
p1.u1ty = 0 ; p1.u1ry = 1 ; p1.u2ty = 0 ; p1.u2ry = 1
p1.v1ty = 0 ; p1.v1ry = 1 ; p1.v2ty = 0 ; p1.v2ry = 1
p1.w1ty = 0 ; p1.w1ry = 1 ; p1.w2ty = 0 ; p1.w2ry = 1

In [78]:
k0_1 = p1.calc_k0()
kG0_1 = p1.calc_kG0()

eigvals, eigvecs = lb(k0_1, kG0_1, silent=True)
eigvals

		Calculating k0... 
		finished!
		Calculating kG0... 
		finished!


d:\programs\Anaconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1593: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "


TypeError: Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.

In [79]:
print(k0_1.min())
print(k0_1.max())

-77.55439975824015
121.49716202529147


In [80]:
print(kG0_1.min())
print(kG0_1.max())

-0.0005555555555555578
0.0005555555555555578


In [95]:
sp.Matrix(kG0_1.toarray())

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                    0, 0, 0, 0, 0, 0,                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                     0, 0, 0, 0, 0, 0,                    0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                    0, 0, 0, 0, 0, 0,                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                     0, 0, 0, 0, 0, 0,                    0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                    0, 0, 0, 0, 0, 0,                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                     0, 0, 0, 0, 0, 0,                    0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                    0, 0, 0, 0, 0, 0,                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                     0, 0, 0, 0, 0, 0,                    0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                    0, 0, 0, 0, 0, 0,       

In [74]:
sp.Matrix(KG)

Matrix([
[0, 0, 0, 0, 0,                     0, 0,                     0, 0, 0, 0, 0, 0,                     0, 0,                     0],
[0, 0, 0, 0, 0,                     0, 0,                     0, 0, 0, 0, 0, 0,                     0, 0,                     0],
[0, 0, 0, 0, 0,                     0, 0,                     0, 0, 0, 0, 0, 0,                     0, 0,                     0],
[0, 0, 0, 0, 0,                     0, 0,                     0, 0, 0, 0, 0, 0,                     0, 0,                     0],
[0, 0, 0, 0, 0,                     0, 0,                     0, 0, 0, 0, 0, 0,                     0, 0,                     0],
[0, 0, 0, 0, 0,  -0.00126984126984127, 0,  0.000595238095238095, 0, 0, 0, 0, 0,  0.000317460317460317, 0, -0.000198412698412697],
[0, 0, 0, 0, 0,                     0, 0,                     0, 0, 0, 0, 0, 0,                     0, 0,                     0],
[0, 0, 0, 0, 0,  0.000595238095238095, 0,  -0.00129870129870129, 0, 0, 0, 0, 0, -

## Check with composipy

In [ ]:
from composipy import Laminate, Ply, buckling_load

In [ ]:
ply = Ply(E1, E2, v12, G12, t)
layup_1 = [(45, ply),(0, ply),(0, ply),(45, ply),(0, ply),(0, ply),(45, ply)]
laminate_1 = Laminate(layup_1)

In [ ]:
load = buckling_load(a, b, laminate_1.D, shape_plot = False , eig = True, n=10)

In [ ]:
a = abs(load['eigen values'])
a.sort()
a

array([4.14270828e+00, 6.21522205e+00, 1.06650914e+01, 1.65710358e+01,
       1.70242153e+01, 1.91554561e+01, 2.45809295e+01, 2.48385030e+01,
       2.52623245e+01, 3.27636136e+01, 3.53001050e+01, 3.72839695e+01,
       4.01613869e+01, 4.26490962e+01, 4.27935413e+01, 4.67661371e+01,
       4.71871495e+01, 5.44108590e+01, 5.59663308e+01, 6.09072064e+01,
       6.62675469e+01, 6.72164889e+01, 6.81862074e+01, 6.93560842e+01,
       7.10330003e+01, 7.64561102e+01, 7.66218673e+01, 8.06194972e+01,
       8.36422449e+01, 8.68293907e+01, 9.38431588e+01, 9.44488356e+01,
       9.59106601e+01, 9.83663621e+01, 9.94734984e+01, 1.00992872e+02,
       1.03585297e+02, 1.06783130e+02, 1.07936385e+02, 1.13012131e+02,
       1.14423916e+02, 1.14554965e+02, 1.25606923e+02, 1.28759733e+02,
       1.31058289e+02, 1.39329386e+02, 1.49151952e+02, 1.52444711e+02,
       1.53256367e+02, 1.55402551e+02, 1.60648911e+02, 1.71172156e+02,
       1.72395090e+02, 1.87014978e+02, 2.01935205e+02, 2.03008923e+02,
      